# Predictive Maintenance in Jet Engines

Oh oh! There can be no more important application of predictive maintenance than for jet engines. For you to have peace when flying, those engines better not be failing anytime soon. Predictive maintenance tries to balance the cost of failures against the cost of maintenance. 

## Contents
- [The Use Case](#The-Use-Case)
- [Success Criteria](#Success-Criteria)
- [AI Solutions](#AI-Solutions)
    - [LSTM](#LSTM)
    - [Denoising](#Denoising)
- [Thoughts](#Thoughts)

## The Use Case

If an airline company does daily maintenance of their plane engines they will soon go bankrupt. Same applies if they never do any maintenance. Their planes may cause fatal accidents result in them going out of business. So the question is, what is the right amout of scheduled maintenance? 


![optimize](images/optimize.jpg)

The ability to predict Remaining Useful Life (RUL) enables the optimization of maintenance costs. However, this requires a robust IoT strategy to collect indicators of failure from the jet engines. Even with good data collection strategy, events like engine failures rarely occur. Simulations can help generate training data as what [NASA did](https://ti.arc.nasa.gov/tech/dash/groups/pcoe/prognostic-data-repository/)

## Success Criteria

A success predictive maintenance model should be able to operate at the optimal point of the trade-off between the maintenance costs and downtime costs. Since the model will be predicting RUL, metrics to optimize are the Mean Squared Error. 

## AI Solution

![nonsymetric](images/nonsymetric.jpg)

In [ ]:
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (12, 9)
mpl.rcParams['axes.grid'] = False

In [ ]:
print("TF Version: ", tf.__version__)
print(tf.config.list_physical_devices('GPU'))

In [ ]:
# os[n] is the n-th operating setting whilst ms[n] is the n-th sensor measurement

names = ["unit", "time","os1", "os2", "os3"] + ["ms{}".format(ms) for ms in range(1,22)]
names

The input are 4 files. File 1 and 3 have simulations done at sea level conditions while file 2 and 4 have non-sea level conditions. File 1 and 2 experience the HPC degradation fault mode whilst file 3 and 4 experience 2 fault modes of HPC degradation and fan degradation. 

In [ ]:
train_dataframe1 = pd.read_csv("data/train_FD001.txt", sep=" ", names=names, index_col=False) 
train_dataframe2 = pd.read_csv("data/train_FD002.txt", sep=" ", names=names, index_col=False) 
train_dataframe3 = pd.read_csv("data/train_FD003.txt", sep=" ", names=names, index_col=False) 
train_dataframe4 = pd.read_csv("data/train_FD004.txt", sep=" ", names=names, index_col=False) 

In [ ]:
test_dataframe1 = pd.read_csv("data/test_FD001.txt", sep=" ", names=names, index_col=False) 
test_dataframe2 = pd.read_csv("data/test_FD002.txt", sep=" ", names=names, index_col=False) 
test_dataframe3 = pd.read_csv("data/test_FD003.txt", sep=" ", names=names, index_col=False) 
test_dataframe4 = pd.read_csv("data/test_FD004.txt", sep=" ", names=names, index_col=False) 

In [ ]:
test_label1 = pd.read_csv("data/RUL_FD001.txt", sep=" ", names=['rul'], index_col=False) 
test_label2 = pd.read_csv("data/RUL_FD002.txt", sep=" ", names=['rul'], index_col=False) 
test_label3 = pd.read_csv("data/RUL_FD003.txt", sep=" ", names=['rul'], index_col=False) 
test_label4 = pd.read_csv("data/RUL_FD004.txt", sep=" ", names=['rul'], index_col=False) 

In [ ]:
# Each file corresponds to a different condition. ONE is for sea level and SIX is probably for about sea level
train_dataframe1["conditions"] = "ONE"
train_dataframe2["conditions"] = "SIX"
train_dataframe3["conditions"] = "ONE"
train_dataframe4["conditions"] = "SIX"

test_dataframe1["conditions"] = "ONE"
test_dataframe2["conditions"] = "SIX"
test_dataframe3["conditions"] = "ONE"
test_dataframe4["conditions"] = "SIX"

test_label1["conditions"] = "ONE"
test_label2["conditions"] = "SIX"
test_label3["conditions"] = "ONE"
test_label4["conditions"] = "SIX"

#The failure modes are difficult to have an attribute for as file 3 and 4 have 2 failure modes
train_dataframe1["file"] = 1
train_dataframe2["file"] = 2
train_dataframe3["file"] = 3
train_dataframe4["file"] = 4

test_dataframe1["file"] = 1
test_dataframe2["file"] = 2
test_dataframe3["file"] = 3
test_dataframe4["file"] = 4

test_label1["file"] = 1
test_label2["file"] = 2
test_label3["file"] = 3
test_label4["file"] = 4

train_df = pd.concat([train_dataframe1, train_dataframe2, train_dataframe3, train_dataframe4])
test_df = pd.concat([test_dataframe1, test_dataframe2, test_dataframe3, test_dataframe4])
label_df = pd.concat([test_label1, test_label2, test_label3, test_label4])

In [ ]:
train_df

In [ ]:
measurements_by_unit = train_df.groupby(['file', 'unit']).size().reset_index(name='measurements')
measurements_by_unit

In [ ]:
units = train_df.groupby(['file'])['unit'].nunique().reset_index(name='units')
units

In [ ]:
min_measurements_by_unit = measurements_by_unit.groupby(['file'])['measurements'].min().reset_index(name='min_measurements')
min_measurements_by_unit

It looks like as the machine approaches failure measurements like ms2, ms3, ms4, ms8, ms9, ms11, ms14, ms15 and ms16 start to increase and ms7, ms12, ms20 and ms21 start to decrease.

In [ ]:
plot_cols = ['os1', 'os2', 'os3', 'ms1', 'ms2', 'ms3', 'ms4', 'ms5', 'ms6','ms7','ms8', 
  'ms9', 'ms10', 'ms11', 'ms12', 'ms13', 'ms14', 'ms15', 'ms16', 'ms17', 'ms18', 'ms19', 'ms20', 'ms21']
plot_features = train_df[(train_df['unit']==3) & (train_df['file']==1)][plot_cols]
plot_features.index = train_df[(train_df['unit']==3) & (train_df['file']==1)]['time']
_ = plot_features.plot(subplots=True)

In [ ]:
plot_features = train_df[(train_df['unit']==3) & (train_df['file']==4)][plot_cols]
plot_features.index = train_df[(train_df['unit']==3) & (train_df['file']==4)]['time']
_ = plot_features.plot(subplots=True)

For files 2 and 4 where the conditions are not sea level, it looks like the failing pattern is not apparent.

In [ ]:
plot_features = train_df[(train_df['unit']==3) & (train_df['file']==2)][plot_cols]
plot_features.index = train_df[(train_df['unit']==3) & (train_df['file']==2)]['time']
_ = plot_features.plot(subplots=True)

In [ ]:
plot_features = train_df[(train_df['unit']==3) & (train_df['file']==3)][plot_cols]
plot_features.index = train_df[(train_df['unit']==3) & (train_df['file']==3)]['time']
_ = plot_features.plot(subplots=True)

In [ ]:
train_df[(train_df['file']==1)].describe().transpose()

In [ ]:
train_df[(train_df['file']==2)].describe().transpose()

In [ ]:
train_df[(train_df['file']==3)].describe().transpose()

In [ ]:
train_df[(train_df['file']==4)].describe().transpose()

In [ ]:
import numpy
import math
from pandas import read_csv
import pandas
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model


seed = 7
batch_size = 100
numpy.random.seed(seed)

train_dataframe = read_csv("data/train_FD001.txt", sep=" ", header=0, engine='python') 

test_dataframe_x = read_csv("data/test_FD001.txt", sep=" ", header=0, engine='python') 
test_dataframe_y = read_csv("data/RUL_FD001.txt", sep=" ", header=0, engine='python')

columns = train_dataframe.columns
train_dataset = train_dataframe.values
test_dataset_x = test_dataframe_x.values
test_dataset_y = test_dataframe_y.values


input_dimensions = train_dataset.shape[1]
print(input_dimensions)

train_x = train_dataset[:,0:input_dimensions-1]
train_y = train_dataset[:,input_dimensions-1]

test_x = test_dataset_x
test_y = test_dataset_y[:,0]

print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

scaler_x = MinMaxScaler(feature_range=(0, 1)).fit(train_x)
scaler_y = MinMaxScaler(feature_range=(0, 1)).fit(train_y)
train_x = scaler_x.transform(train_x)
train_y = scaler_y.transform(train_y)
test_x = scaler_x.transform(test_x)
test_y = scaler_y.transform(test_y)

print(train_x.shape)


#develop autoencoder model
model = Sequential()
model.add(Dense(input_dimensions-1, input_dim=input_dimensions-1, kernel_initializer='random_normal', activation='sigmoid'))
model.add(Dense(4, kernel_initializer='random_normal', activation='sigmoid'))
model.add(Dense(input_dimensions-1))
# Compile model
sgd = SGD(lr=0.01, momentum=0.1, decay=0.0, nesterov=False)
model.compile(loss='mean_squared_error', optimizer='adadelta', metrics=['accuracy'])

model.fit(train_x, train_x, epochs=200, batch_size=batch_size, verbose=2, shuffle=True)

train_predictions = model.predict(train_x)
pandas.DataFrame(train_predictions, columns = columns[0:input_dimensions-1]).to_csv(path_or_buf='data/autoencoded.csv')


#develop predictive model
model = Sequential()
model.add(Dense(512, input_dim=input_dimensions-1, kernel_initializer='random_normal', activation='relu'))
model.add(Dense(128, kernel_initializer='random_normal', activation='relu'))
model.add(Dense(32, kernel_initializer='random_normal', activation='relu'))
model.add(Dense(16, kernel_initializer='random_normal', activation='relu'))
model.add(Dense(1))
# Compile model
model.compile(loss='mean_squared_error', optimizer='adadelta', metrics=['accuracy'])

#use denoised values from autoencoder to train model
model.fit(train_predictions, train_y, epochs=2000, batch_size=batch_size, verbose=2, shuffle=True)

train_predictions = model.predict(train_x)
test_predictions = model.predict(test_x)

# invert predictions
train_predictions = scaler_y.inverse_transform(train_predictions)
train_y = scaler_y.inverse_transform([train_y])
test_predictions = scaler_y.inverse_transform(test_predictions)
test_y = scaler_y.inverse_transform([test_y])
# calculate mean squared error

train_score = math.sqrt(mean_squared_error(train_y[0], train_predictions[:,0]))
print('Train Score: %.4f RMSE' % (train_score))
test_rmse = math.sqrt(mean_squared_error(test_y[0], test_predictions[:,0]))
print('Test Score: %.4f RMSE' % (test_rmse))
test_mae = mean_absolute_error(test_y[0], test_predictions[:,0])
print('Test Score: %.4f MAE' % (test_mae))

print ('Mean yields actual: %.3f vs predicted: %.3f' % (numpy.mean(test_y[0]), numpy.mean(test_predictions[:,0])))
test_actual_vs_predictions = pandas.concat([pandas.DataFrame(test_y[0], columns=['actual']).reset_index(drop=True), pandas.DataFrame(test_predictions[:,0], columns = ['predicted'])], axis=1)
test_actual_vs_predictions.to_csv(path_or_buf='data/predicted.csv')
